In [144]:
pip install pandas numpy statsmodels openpyxl


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [145]:
# Importando as bibliotecas necessárias
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [146]:
# Importando a base de dados lista3.xlsx
df = pd.read_excel("lista3.xlsx")
# Visualizando as primeiras linhas do DataFrame
print(df.head())

   municipio  seca      pol_A       pol_B  pol_C        IMR    pobreza
0          1     0  44.581902  127.601456      1   9.511067  61.650898
1          2     0  38.821281  115.130013      1  10.622714  62.021793
2          3     0  46.186981  123.228416      0   8.318838  43.510490
3          4     0  33.826748  126.165230      0   7.702812  45.701168
4          5     1  42.859356  120.804451      1  13.184786  61.225948


In [147]:
# Estatísticas básicas
print(df[['pol_A', 'pol_B', 'pol_C', 'pobreza', 'seca']].describe())

             pol_A        pol_B        pol_C      pobreza         seca
count  3000.000000  3000.000000  3000.000000  3000.000000  3000.000000
mean     40.108624   120.088482     0.499000    50.085481     0.291667
std       9.924145     4.994582     0.500082     9.936559     0.454605
min       9.448576   102.599785     0.000000    18.467354     0.000000
25%      33.488738   116.666901     0.000000    43.484351     0.000000
50%      39.918070   120.235359     0.000000    49.986052     0.000000
75%      46.959776   123.440245     1.000000    56.963622     1.000000
max      72.369392   137.068008     1.000000    88.440125     1.000000


In [148]:
# Correlação com pobreza
print(df[['pol_A', 'pol_B', 'pol_C', 'pobreza','seca']].corr())

            pol_A     pol_B     pol_C   pobreza      seca
pol_A    1.000000 -0.015824  0.009938  0.011652 -0.009991
pol_B   -0.015824  1.000000 -0.002622  0.006736  0.000730
pol_C    0.009938 -0.002622  1.000000  0.801651 -0.069853
pobreza  0.011652  0.006736  0.801651  1.000000 -0.075387
seca    -0.009991  0.000730 -0.069853 -0.075387  1.000000


In [149]:
# Regressão da política sobre pobreza
for pol in ['pol_A', 'pol_B', 'pol_C']:
    model = smf.ols(f'{pol} ~ pobreza', data=df).fit()
    print(f"\nRegressão: {pol} ~ pobreza")
    print(model.summary())


Regressão: pol_A ~ pobreza
                            OLS Regression Results                            
Dep. Variable:                  pol_A   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.4071
Date:                Sun, 26 Oct 2025   Prob (F-statistic):              0.524
Time:                        17:00:57   Log-Likelihood:                -11141.
No. Observations:                3000   AIC:                         2.229e+04
Df Residuals:                    2998   BIC:                         2.230e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     39.5258   

In [150]:
import statsmodels.formula.api as smf

# (1) IMR ~ pol_A
reg1 = smf.ols('IMR ~ pol_A', data=df).fit()

# (2) IMR ~ pol_B
reg2 = smf.ols('IMR ~ pol_B', data=df).fit()

# (3) IMR ~ pol_C
reg3 = smf.ols('IMR ~ pol_C', data=df).fit()

# (4) IMR ~ pol_A + pol_B + pol_C
reg4 = smf.ols('IMR ~ pol_A + pol_B + pol_C', data=df).fit()

# (5) IMR ~ pol_A + pol_B + pol_C + pobreza
reg5 = smf.ols('IMR ~ pol_A + pol_B + pol_C + pobreza', data=df).fit()

# (6) IMR ~ pol_A + pol_B + pol_C + pobreza + seca
reg6 = smf.ols('IMR ~ pol_A + pol_B + pol_C + pobreza + seca', data=df).fit()

# (7) IMR ~ pol_A + pol_B + pol_C + pobreza + seca
reg7 = smf.ols('IMR ~ seca', data=df).fit()

# Visualizar resultados principais (coeficientes, p-valores e R²)
for i, reg in enumerate([reg1, reg2, reg3, reg4, reg5, reg6, reg7], start=1):
    print(f"\n--- Regressão ({i}) ---")
    print(reg.summary())


--- Regressão (1) ---
                            OLS Regression Results                            
Dep. Variable:                    IMR   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     57.05
Date:                Sun, 26 Oct 2025   Prob (F-statistic):           5.61e-14
Time:                        17:00:57   Log-Likelihood:                -6359.1
No. Observations:                3000   AIC:                         1.272e+04
Df Residuals:                    2998   BIC:                         1.273e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     10.5760      0.

In [151]:
import pandas as pd
import numpy as np

# Função para formatar p-values (até 3 casas decimais)
def formatar_pvalor(p):
    if p < 0.001:
        return "<0.001"
    else:
        return f"{p:.3f}"

# Função para resumir modelos com arredondamento
def resumir_modelo(modelo, nome):
    coef = modelo.params.round(4)
    pvals = modelo.pvalues.apply(formatar_pvalor)
    r2 = round(modelo.rsquared, 3)

    return pd.DataFrame({
        'modelo': nome,
        'coef': coef,
        'p-valor': pvals,
        'R²': r2
    })

# Juntando todos os modelos
tabela_resumo = pd.concat([
    resumir_modelo(reg1, '(1) IMR ~ pol_A'),
    resumir_modelo(reg2, '(2) IMR ~ pol_B'),
    resumir_modelo(reg3, '(3) IMR ~ pol_C'),
    resumir_modelo(reg4, '(4) + pol_A + pol_B + pol_C'),
    resumir_modelo(reg5, '(5) + pobreza'),
    resumir_modelo(reg6, '(6) + pobreza + seca'),
    resumir_modelo(reg7, '(7) IMR ~ pol_C')

])

print(tabela_resumo)

                                modelo     coef p-valor     R²
Intercept              (1) IMR ~ pol_A  10.5760  <0.001  0.019
pol_A                  (1) IMR ~ pol_A  -0.0280  <0.001  0.019
Intercept              (2) IMR ~ pol_B  15.9019  <0.001  0.017
pol_B                  (2) IMR ~ pol_B  -0.0537  <0.001  0.017
Intercept              (3) IMR ~ pol_C   9.2619  <0.001  0.009
pol_C                  (3) IMR ~ pol_C   0.3816  <0.001  0.009
Intercept  (4) + pol_A + pol_B + pol_C  16.9542  <0.001  0.046
pol_A      (4) + pol_A + pol_B + pol_C  -0.0286  <0.001  0.046
pol_B      (4) + pol_A + pol_B + pol_C  -0.0545  <0.001  0.046
pol_C      (4) + pol_A + pol_B + pol_C   0.3858  <0.001  0.046
Intercept                (5) + pobreza  15.2226  <0.001  0.062
pol_A                    (5) + pobreza  -0.0288  <0.001  0.062
pol_B                    (5) + pobreza  -0.0553  <0.001  0.062
pol_C                    (5) + pobreza  -0.3062   0.011  0.062
pobreza                  (5) + pobreza   0.0434  <0.001

In [152]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np

# Função para formatar saída
def mostrar_resultados(nome, modelo):
    print(f"\n--- {nome} ---")
    resultados = pd.DataFrame({
        "coef": modelo.params.round(4),
        "p-valor": modelo.pvalues.apply(lambda p: "<0.001" if p < 0.001 else round(p, 3))
    })
    print(resultados)
    print("R² =", round(modelo.rsquared, 4))

# (1) IMR ~ pol_C
reg1 = smf.ols('IMR ~ pol_C', data=df).fit()
mostrar_resultados("(1) IMR ~ pol_C", reg1)

# (2) IMR ~ pobreza
reg2 = smf.ols('IMR ~ pobreza', data=df).fit()
mostrar_resultados("(2) IMR ~ pobreza", reg2)

# (3) IMR ~ pol_C + pobreza
reg3 = smf.ols('IMR ~ pol_C + pobreza', data=df).fit()
mostrar_resultados("(3) IMR ~ pol_C + pobreza", reg3)

# (4) pol_C ~ pobreza e salvar resíduos
reg4 = smf.ols('pol_C ~ pobreza', data=df).fit()
df['pol_C_resid'] = reg4.resid
mostrar_resultados("(4) pol_C ~ pobreza", reg4)

print("\n(4) Resíduos de pol_C ~ pobreza criados: df['pol_C_resid']")

# (5) IMR ~ pol_C_resid
reg5 = smf.ols('IMR ~ pol_C_resid', data=df).fit()
mostrar_resultados("(5) IMR ~ pol_C_resid", reg5)

# Comparação entre (3) e (5)
print("\nComparação direta dos coeficientes da Política C:")
print(" β̂_pol_C (regressão 3)      =", round(reg3.params['pol_C'], 4))
print(" β̂_pol_C_resid (regressão 5) =", round(reg5.params['pol_C_resid'], 4))


--- (1) IMR ~ pol_C ---
             coef p-valor
Intercept  9.2619  <0.001
pol_C      0.3816  <0.001
R² = 0.0088

--- (2) IMR ~ pobreza ---
             coef p-valor
Intercept  7.9212  <0.001
pobreza    0.0306  <0.001
R² = 0.0223

--- (3) IMR ~ pol_C + pobreza ---
             coef p-valor
Intercept  7.4727  <0.001
pol_C     -0.2947   0.016
pobreza    0.0425  <0.001
R² = 0.0242

--- (4) pol_C ~ pobreza ---
             coef p-valor
Intercept -1.5217  <0.001
pobreza    0.0403  <0.001
R² = 0.6426

(4) Resíduos de pol_C ~ pobreza criados: df['pol_C_resid']

--- (5) IMR ~ pol_C_resid ---
               coef p-valor
Intercept    9.4523  <0.001
pol_C_resid -0.2947   0.018
R² = 0.0019

Comparação direta dos coeficientes da Política C:
 β̂_pol_C (regressão 3)      = -0.2947
 β̂_pol_C_resid (regressão 5) = -0.2947


In [153]:
# (i) Dividindo por 100
df['pol_B_100'] = df['pol_B'] / 100

# (ii) Centralizando (subtraindo a média de pol_B)
df['pol_B_media'] = df['pol_B'] - df['pol_B'].mean()

# (iii) Criar IMR_log = ln(IMR)
df['IMR_log'] = np.log(df['IMR'])

# (iv) Criar pobreza_log = ln(pobreza)
df['pobreza_log'] = np.log(df['pobreza'])

# Visualizar as primeiras linhas do DataFrame modificado
print(df.head())

   municipio  seca      pol_A       pol_B  pol_C        IMR    pobreza  \
0          1     0  44.581902  127.601456      1   9.511067  61.650898   
1          2     0  38.821281  115.130013      1  10.622714  62.021793   
2          3     0  46.186981  123.228416      0   8.318838  43.510490   
3          4     0  33.826748  126.165230      0   7.702812  45.701168   
4          5     1  42.859356  120.804451      1  13.184786  61.225948   

   pol_C_resid  pol_B_100  pol_B_media   IMR_log  pobreza_log  
0     0.034392   1.276015     7.512974  2.252456     4.121488  
1     0.019428   1.151300    -4.958469  2.362995     4.127486  
2    -0.233731   1.232284     3.139935  2.118523     3.773002  
3    -0.322114   1.261652     6.076748  2.041585     3.822124  
4     0.051536   1.208045     0.715969  2.579064     4.114571  


In [155]:
import statsmodels.formula.api as smf

# (1) IMR ~ pol_B
reg1 = smf.ols('IMR ~ pol_B', data=df).fit()

# (2) IMR ~ pol_B_100
reg2 = smf.ols('IMR ~ pol_B_100', data=df).fit()

# (3) IMR ~ pol_B_media
reg3 = smf.ols('IMR ~ pol_B_media', data=df).fit()

# (4) ln(IMR) ~ pol_B
reg4 = smf.ols('IMR_log ~ pol_B', data=df).fit()

# (5) ln(IMR) ~ ln(pobreza)
reg5 = smf.ols('IMR_log ~ pobreza_log', data=df).fit()

# Mostrar resultados (coeficiente, p-valor, R²):
def resumo(m, nome):
    print(f"\n=== {nome} ===")
    print(m.summary().tables[1])  # tabela com coeficientes
    print("R²:", round(m.rsquared, 4))

resumo(reg1, "(1) IMR ~ pol_B")
resumo(reg2, "(2) IMR ~ pol_B_100")
resumo(reg3, "(3) IMR ~ pol_B_media")
resumo(reg4, "(4) IMR_log ~ pol_B")
resumo(reg5, "(5) IMR_log ~ pobreza_log")


=== (1) IMR ~ pol_B ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.9019      0.886     17.939      0.000      14.164      17.640
pol_B         -0.0537      0.007     -7.282      0.000      -0.068      -0.039
R²: 0.0174

=== (2) IMR ~ pol_B_100 ===
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     15.9019      0.886     17.939      0.000      14.164      17.640
pol_B_100     -5.3707      0.738     -7.282      0.000      -6.817      -3.925
R²: 0.0174

=== (3) IMR ~ pol_B_media ===
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       9.4523      0.037    256.645      0.000       9.380       9.525
pol_B_media    -0.053

In [158]:
import statsmodels.formula.api as smf
import numpy as np

# 1. Criar dummies de interação entre políticas e seca (se ainda não existirem):
df['polA_seca'] = df['pol_A'] * df['seca']
df['polB_seca'] = df['pol_B'] * df['seca']
df['polC_seca'] = df['pol_C'] * df['seca']

# 2. Criar os termos de não linearidade para pobreza:
df['pobreza2'] = df['pobreza']**2
df['pobreza3'] = df['pobreza']**3

In [160]:
df.head()  # Visualizar as primeiras linhas do DataFrame modificado

,municipio,seca,pol_A,pol_B,pol_C,IMR,pobreza,pol_C_resid,pol_B_100,pol_B_media,IMR_log,pobreza_log,polA_seca,polB_seca,polC_seca,pobreza2,pobreza3
0,1,0,44.581902,127.601456,1,9.511067,61.650898,0.034392,1.276015,7.512974,2.252456,4.121488,0.000000,0.000000,0,3800.833222,234324.781190
1,2,0,38.821281,115.130013,1,10.622714,62.021793,0.019428,1.151300,-4.958469,2.362995,4.127486,0.000000,0.000000,0,3846.702852,238579.409442
2,3,0,46.186981,123.228416,0,8.318838,43.510490,-0.233731,1.232284,3.139935,2.118523,3.773002,0.000000,0.000000,0,1893.162776,82372.440840
3,4,0,33.826748,126.165230,0,7.702812,45.701168,-0.322114,1.261652,6.076748,2.041585,3.822124,0.000000,0.000000,0,2088.596762,95451.311634
4,5,1,42.859356,120.804451,1,13.184786,61.225948,0.051536,1.208045,0.715969,2.579064,4.114571,42.859356,120.804451,1,3748.616749,229512.615420


In [161]:
# (1) IMR ~ seca
reg1 = smf.ols('IMR ~ seca', data=df).fit()

# (2) IMR ~ pol_A + pol_B + pol_C + seca + pobreza
reg2 = smf.ols('IMR ~ pol_A + pol_B + pol_C + seca + pobreza', data=df).fit()

# (3) IMR ~ pol_A + pol_B + pol_C + pobreza + seca + interações
reg3 = smf.ols('IMR ~ pol_A + pol_B + pol_C + pobreza + seca + polA_seca + polB_seca + polC_seca', data=df).fit()

# (4) IMR_log ~ pobreza + pobreza² + pobreza³ + seca
reg4 = smf.ols('IMR_log ~ pobreza + pobreza2 + pobreza3 + seca', data=df).fit()

In [162]:
import pandas as pd

def resumo(modelo, nome):
    print(f"\n================ {nome} ================")
    print(modelo.summary().tables[1])  # coeficientes
    print("R² =", round(modelo.rsquared, 4))

resumo(reg1, "Regressão (1) IMR ~ seca")
resumo(reg2, "Regressão (2) IMR ~ pol_A + pol_B + pol_C + seca + pobreza")
resumo(reg3, "Regressão (3) IMR ~ políticas + pobreza + seca + interações pol*seca")
resumo(reg4, "Regressão (4) ln(IMR) ~ pobreza + pobreza² + pobreza³ + seca")


================ Regressão (1) IMR ~ seca ================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      8.4854      0.030    286.083      0.000       8.427       8.544
seca           3.3150      0.055     60.360      0.000       3.207       3.423
R² = 0.5486

================ Regressão (2) IMR ~ pol_A + pol_B + pol_C + seca + pobreza ================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.7540      0.576     23.858      0.000      12.624      14.884
pol_A         -0.0274      0.002    -11.985      0.000      -0.032      -0.023
pol_B         -0.0555      0.005    -12.225      0.000      -0.064      -0.047
pol_C         -0.2253      0.076     -2.969      0.003      -0.374      -0.077
seca           3.3776      0.050     67.475 

In [163]:
df.groupby('seca')['IMR'].mean()

seca
0     8.485437
1    11.800484
Name: IMR, dtype: float64